In [7]:
# -*- coding: utf-8 -*-
import os
import keras
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D, Lambda, Input, Dense, Flatten, BatchNormalization
from keras.models import Model
from keras.layers.core import Dropout
from keras import optimizers
from keras import backend 
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau,TensorBoard

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.preprocessing import OneHotEncoder
from keras.datasets import cifar10
import cv2
# import gc
import numpy as np

print(tf.__version__)
print(keras.__version__)

1.13.0-dev20181104
2.2.4


In [8]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

###kerasのバックエンドで使用するライブラリにより入力チャネルの順番が異なる為、条件分岐
if backend.image_data_format() == 'channels_first':
    inputs = Input(shape = (1, 28, 28))
    x_train = x_train.reshape(x_train.shape[0], 1,28, 28).astype('float32')/255.
    x_test = x_test.reshape(x_test.shape[0], 1, 28, 28).astype('float32')/255
else:
    inputs = Input(shape= (28, 28, 1))
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')/255.
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')/255
    
    # convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [19]:
inputs = Input(shape=(28, 28, 1))
# Due to memory limitation, images will resized on-the-fly.
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='block1_pool')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='block2_pool')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='block3_pool')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='block4_pool')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), padding='same', name='block5_pool')(x)
flattened = Flatten(name='flatten')(x)
x = Dense(128, activation='relu', name='fc1')(flattened)
x = Dropout(0.5, name='dropout1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
x = Dropout(0.5, name='dropout2')(x)
# CIFAR10は10クラスなので出力の数が違う
predictions = Dense(10, activation='softmax', name='predictions')(x)


BATCH_SIZE = 128
sgd = optimizers.SGD(lr=0.01,
                     momentum=0.9,
                     decay=5e-4)#, nesterov=False)

model = Model(inputs=inputs, outputs=predictions)


model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 28, 28, 64)        640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 28, 28, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 14, 14, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 14, 14, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 14, 14, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 7, 7, 128)         0         
__________

In [18]:
%%time 
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ[" CUDA_VISIBLE_DEVICES "] = "0"

# Train
rlop = ReduceLROnPlateau(monitor='val_acc',
                         factor=0.1,
                         patience=5,
                         verbose=1,
                         mode='auto',
                         cooldown=0,
                         min_lr=0.00001)

 
model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=20, verbose=1,
              callbacks=[rlop], validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20


ResourceExhaustedError: OOM when allocating tensor of shape [512] and type float
	 [[node block4_conv1_3/Const (defined at /home/ksatake/.conda/envs/py3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:430) ]]

Caused by op 'block4_conv1_3/Const', defined at:
  File "/home/ksatake/.conda/envs/py3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ksatake/.conda/envs/py3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ksatake/.conda/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ksatake/.conda/envs/py3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ksatake/.conda/envs/py3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/ksatake/.conda/envs/py3/lib/python3.6/site-

In [12]:
plt.subplot(2, 2, 1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(2, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.show()

NameError: name 'plt' is not defined